In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## Step 1: creating a file with critical verbs and their RNN surprisal values

In [3]:
baseloc = "C:\\Users\\rondin\\Desktop\\Courses\\Computational Psycholinguistics\\Assignment\\Assignment 2"

In [34]:
ambiguity = ["amb","unamb"]
fit = ["good", "bad"]

df_verbs = pd.DataFrame()
dfs = pd.DataFrame()
for amb in ambiguity:
    for f in fit:
        
        df = pd.DataFrame()
        
        # read surprisal from file
        file_lstm = baseloc + "\\surprisal.lstm.{}_{}_fit_new.csv".format(amb,f)
        file_srn = baseloc + "\\surprisal.srn.{}_{}_fit_new.csv".format(amb,f)
        df_lstm = pd.read_csv(file_lstm)
        df_srn = pd.read_csv(file_srn)
        
        # add surprisal from both models into one single df
        df["sent_nr"] = df_lstm["sent_nr"]
        df["word_pos"] = df_lstm["word_pos"]
        df["word"] = df_lstm["word"]
        df["surp_lstm"] = df_lstm["LSTM_1_8773568"]
        df["surp_srn"] = df_srn["SRN_1_8773568"]
        df["ambiguity"] = amb
        df["fit"] = f
        
        # find the critical verbs
        and_s = df[df["word"] == "and"]
        
        sent_nr = and_s["sent_nr"].to_list()
        word_pos_and = and_s["word_pos"].to_list()
        word_pos_verb = [pos + 3 for pos in word_pos_and]
        
        verb_s = pd.DataFrame()
        for i in range(len(sent_nr)):
            verb_s = verb_s.append(df[(df["sent_nr"] == sent_nr[i]) & (df["word_pos"] == word_pos_verb[i])])
        
        dfs = dfs.append(df) # save the df with all words in case there's any issue with the verb df/file
    
        df_verbs = df_verbs.append(verb_s)

df_verbs = df_verbs.reset_index()

In [39]:
len(dfs)

3793

In [36]:
# save the verb df/file
df_verbs.to_csv("verbs_first_version.csv")

The saved file then goes through a visual check to see if any word other than critical verbs is mistakenly included. It turns out that indeed a few 3-word long second objects and verb-preceding adjectives have been included. Those cases have thus been marked and changes to the corresponding critical verbs are made accordingly.

In [53]:
# to find the correct target critical verbs during the check
dfs[(dfs["sent_nr"] == 44) & (dfs["word_pos"] == 10)]

,sent_nr,word_pos,word,surp_lstm,surp_srn,ambiguity,fit
691,44,10,aquarium,10.037937,10.521035,amb,good
689,44,10,dinner,8.062805,8.902812,amb,bad
735,44,10,the,2.033485,2.378748,unamb,good
733,44,10,made,9.683013,8.066246,unamb,bad


## Step 2: building the file for LMER analyses

In [4]:
# reload the verb file
file_v = baseloc + "\\verbs_first_version.csv"
df_verbs = pd.read_csv(file_v)
df_verbs.head()

,Unnamed: 0,index,sent_nr,word_pos,word,surp_lstm,surp_srn,ambiguity,fit,isPrecededbyAdv
0,0,8,0,8,spotted,10.149651,9.466072,amb,good,NaN
1,1,24,1,9,washed,8.380904,10.168794,amb,good,NaN
2,2,37,2,8,buries,13.338874,14.105397,amb,good,NaN
3,3,52,3,8,films,11.397077,12.100304,amb,good,NaN
4,4,69,4,9,scolded,10.857853,12.349148,amb,good,1.0


In [7]:
# discard the trials with an adjective preceding the critical verb
df = df_verbs.drop(df_verbs[df_verbs["isPrecededbyAdv"] == 1].index)
len(df[(df["ambiguity"] == "amb") & (df["fit"] == "bad")])

58

In [65]:
# melt lstm and srn surprisal into two levels of a variable for the LMER model which contrasts the two models' performance
colnames = df.columns.tolist()[5:7]
df_melted = pd.melt(df, id_vars=["word","word_pos","ambiguity","fit"], value_vars=colnames, var_name='model', value_name='surprisal')
for i in range(len(df_melted)):
    if df_melted["model"][i] == "surp_lstm":
        df_melted["model"][i] = "lstm"
    else:
        df_melted["model"][i] = "srn"

df_melted.head()

<ipython-input-65-6f46eb234520>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_melted["model"][i] = "lstm"
<ipython-input-65-6f46eb234520>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_melted["model"][i] = "srn"


,word,word_pos,ambiguity,fit,model,surprisal
0,spotted,8,amb,good,lstm,10.149651
1,washed,9,amb,good,lstm,8.380904
2,buries,8,amb,good,lstm,13.338874
3,films,8,amb,good,lstm,11.397077
4,complained,8,amb,good,lstm,8.378635


In [68]:
# save the datafile
df_melted.to_csv("datafile_p2.csv")